In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.4 MB/s eta 0:00:00


# Download Video Function

In [ ]:
import os
from pytube import YouTube

def Download(link):
    youtubeObject = YouTube(link)
    youtubeObject = youtubeObject.streams.get_highest_resolution()
    try:
        video_file_path = youtubeObject.download()
        print("Download is completed successfully")

    except Exception as e:
        print(f"An error has occurred: {e}")


In [ ]:
if __name__ == "__main__":
    link = input("Enter the YouTube video URL: ")
    Download(link)

Enter the YouTube video URL: https://youtu.be/U3PerKPQM0E\
Download is completed successfully


# Convert to WAV Function

In [ ]:
!pip install pydub

In [ ]:
from os import path
from pydub import AudioSegment

def convertwav(src, dst):
  sound = AudioSegment.from_file(src,format="mp4")
  sound.export(dst, format="wav")

src = "/content/Microsoft (MSFT) Q1 2023 Earnings Call.mp4"
dst = "test1.wav"
convertwav(src, dst)

In [ ]:
import wave
import os

def split_wav(file_path, chunk_size, output_dir):
    with wave.open(file_path, 'rb') as wav:
        params = wav.getparams()
        frames_per_chunk = chunk_size * params.framerate * params.nchannels
        num_chunks = (wav.getnframes() + frames_per_chunk - 1) // frames_per_chunk

        for i in range(num_chunks):
            output_file = os.path.join(output_dir, f"chunk_{i+1}.wav")
            with wave.open(output_file, 'wb') as chunk:
                chunk.setparams(params)
                chunk.writeframes(wav.readframes(int(frames_per_chunk)))

# WAV to Words

In [ ]:
!pip install openai
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 20.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_c

In [ ]:
import whisper

model = whisper.load_model("large-v2")
result = model.transcribe("/content/test1.wav")
print(result["text"])

100%|█████████████████████████████████████| 2.87G/2.87G [05:22<00:00, 9.56MiB/s]


 guten tart Rye Flour Electrician's Brandeis Islands Shells Almond Greetings and welcome to the Microsoft Fiscal Year 2023 First Class of 2020. Welcome to the Microsoft Fiscal Year 2023 First Class of 2020. Welcome to the Microsoft Fiscal Year 2023 First Class of 2020. And thank you for joining us today. On the call with me are Satya Nadella, Chairman and Chief Executive Officer, Amy Hood, Chief Financial Officer, Alice Jollett, Chief Accounting Officer, and Keith Doliver, Deputy General Counsel. On the Microsoft Investor Relations website, you can find our earnings press release and financial summary slide deck, which is intended to supplement our prepared remarks during today's call and provides the reconciliation of differences between GAAP and non-GAAP financial measures. On this call, we will discuss certain non-GAAP items. The non-GAAP financial measures provided should not be considered as a substitute for or superior to the measures of financial performance prepared in accordan

# BART Summarizer

In [ ]:
import pandas as pd
import random
from transformers import BartTokenizer, BartForConditionalGeneration
from bert_score import score
from rouge_score import rouge_scorer

# Load the trained model
model_dir = "/content/drive/MyDrive/TrainingBart/FineTuned"
tokenizer = BartTokenizer.from_pretrained(model_dir)
model = BartForConditionalGeneration.from_pretrained(model_dir)

In [ ]:
def chunk_text(text, chunk_size=1024):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

def summarize_chunk(chunk):
    inputs = tokenizer.encode("summarize: " + chunk, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
chunks = chunk_text(transcript, chunk_size=1024)
chunk_summaries = [summarize_chunk(chunk) for chunk in chunks]

combined_summary = ' '.join(chunk_summaries)

P, R, F1 = score([combined_summary], [transcript], lang="en", verbose=True)

rouge_scores = rouge_scorer.score(transcript, combined_summary)

print(f"Summary: {combined_summary}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")
print(f"ROUGE - ROUGE-1: {rouge_scores['rouge1'].fmeasure}, ROUGE-2: {rouge_scores['rouge2'].fmeasure}, ROUGE-L: {rouge_scores['rougeL'].fmeasure}")
print("-------------------------------------------------------------")

#FINGPT Model

In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 28.7 MB/s eta 0:00:00


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("FinGPT/fingpt-forecaster_dow30_llama2-7b_lora")
base_model = AutoModelForCausalLM.from_pretrained("base_models/Llama-2-7b-chat-hf")
model = PeftModel.from_pretrained(base_model, "FinGPT/fingpt-forecaster_dow30_llama2-7b_lora")

ModuleNotFoundError: No module named 'peft'